In [1]:
from scipy.io import netcdf
import numpy
import netCDF4
import pandas
import aacgmv2
%pylab inline

/usr/local/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


Populating the interactive namespace from numpy and matplotlib


In [2]:
ampFName = "../data/20110516.0000.86400.120.north.grd.ncdf"

In [6]:
currDataSet = netCDF4.Dataset(ampFName)
currDataSet.variables.keys()

[u'nlon',
 u'nlat',
 u'start_yr',
 u'start_mo',
 u'start_dy',
 u'start_hr',
 u'start_mt',
 u'start_sc',
 u'end_yr',
 u'end_mo',
 u'end_dy',
 u'end_hr',
 u'end_mt',
 u'end_sc',
 u'colat',
 u'mlt',
 u'dBnorth1',
 u'dBeast1',
 u'dBnorth2',
 u'dBeast2',
 u'ddBnorth1',
 u'ddBeast1',
 u'ddBnorth2',
 u'ddBeast2',
 u'dBr',
 u'ddBr',
 u'Jr',
 u'dJr',
 u'dBprt']

In [23]:
currDataSet.variables["colat"][:]

array([[  1.,   2.,   3., ...,  48.,  49.,  50.],
       [  1.,   2.,   3., ...,  48.,  49.,  50.],
       [  1.,   2.,   3., ...,  48.,  49.,  50.],
       ..., 
       [  1.,   2.,   3., ...,  48.,  49.,  50.],
       [  1.,   2.,   3., ...,  48.,  49.,  50.],
       [  1.,   2.,   3., ...,  48.,  49.,  50.]], dtype=float32)